## Observations and Insights 

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files (I did something janky with Git/Github and had to 
#patch the files into an upload/download situation and so the names are weird.)

mouse_metadata_path = "05-Matplotlib_Homework_Instructions_Pymaceuticals_data_Mouse_metadata.csv"
study_results_path = "05-Matplotlib_Homework_Instructions_Pymaceuticals_data_Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

mouse_metadata_df=pd.DataFrame(mouse_metadata)
study_results_df=pd.DataFrame(study_results)

# Combine the data into a single dataset

combined_df=mouse_metadata_df.append(study_results_df)

# Display the data table for preview
combined_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21.0,16.0,NaN,NaN,NaN
1,s185,Capomulin,Female,3.0,17.0,NaN,NaN,NaN
2,x401,Capomulin,Female,16.0,15.0,NaN,NaN,NaN
3,m601,Capomulin,Male,22.0,17.0,NaN,NaN,NaN
4,g791,Ramicane,Male,11.0,16.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1888,r944,NaN,NaN,NaN,NaN,45.0,41.581521,2.0
1889,u364,NaN,NaN,NaN,NaN,45.0,31.023923,3.0
1890,p438,NaN,NaN,NaN,NaN,45.0,61.433892,1.0
1891,x773,NaN,NaN,NaN,NaN,45.0,58.634971,4.0


In [53]:
# Checking the number of mice.
unique_mice=combined_df['Mouse ID'].unique()
unique_mice_df=pd.DataFrame(unique_mice)
unique_mice_df


,0
0,k403
1,s185
2,x401
3,m601
4,g791
...,...
244,z314
245,z435
246,z581
247,z795


In [56]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
#duplicate_mice=combined_df.groupby(['Mouse ID', "Timepoint"])
#duplicate_mice_df=pd.DataFrame(duplicate_mice)

#duplicate_mice_df

duplicate_mouse = combined_df.loc[combined_df.duplicated(subset = ["Mouse ID", "Timepoint"]), "Mouse ID"]
duplicate_mouse


137     g989
360     g989
681     g989
869     g989
1111    g989
Name: Mouse ID, dtype: object

In [59]:
# Optional: Get all the data for the duplicate mouse ID.
duplicate_mouse_data=combined_df[combined_df["Mouse ID"]=='g989']
duplicate_mouse_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
107,g989,Propriva,Female,21.0,26.0,NaN,NaN,NaN
107,g989,NaN,NaN,NaN,NaN,0.0,45.000000,0.0
137,g989,NaN,NaN,NaN,NaN,0.0,45.000000,0.0
329,g989,NaN,NaN,NaN,NaN,5.0,48.786801,0.0
360,g989,NaN,NaN,NaN,NaN,5.0,47.570392,0.0
620,g989,NaN,NaN,NaN,NaN,10.0,51.745156,0.0
681,g989,NaN,NaN,NaN,NaN,10.0,49.880528,0.0
815,g989,NaN,NaN,NaN,NaN,15.0,51.325852,1.0
869,g989,NaN,NaN,NaN,NaN,15.0,53.442020,0.0
950,g989,NaN,NaN,NaN,NaN,20.0,55.326122,1.0


In [4]:


 

#popular_items=playerdata.groupby(['Item ID', 'Item Name']).sum()["Price"].rename("Total Purchase Value")
#duplicate_mice_df["Timepoint"]=combined_df["Timepoint"]
combined_df[["Mouse ID", "Timepoint"]]=id_timepoint

id_timepoint_df=pd.DataFrame(id_timepoint)
id_timepoint_df_sorted=id_timepoint_df.sort_values(["Timepoint"], ascending=False)

id_timepoint_df_sorted


In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.


In [6]:
# Checking the number of mice in the clean DataFrame.


## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.



In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
